In [1]:
import os
import pandas as pd
import numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.common.by import By

## 1) Tor-Selenium Setup

In [7]:
torexe = os.popen(r'C:\Users\admin\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe')

firefoxBinary = r"C:\Users\admin\Desktop\Tor Browser\Browser\firefox.exe"
geckodriverPath = r"C:\Users\admin\Desktop\Tor Browser\Browser\geckodriver.exe"
proxyIP = "127.0.0.1"
proxyPort = 9150

binary = FirefoxBinary(firefoxBinary)

proxy_settings = {
    "network.proxy.type": 1,
    "network.proxy.socks": proxyIP,
    "network.proxy.socks_port": proxyPort,
    "network.proxy.socks_remote_dns": False,
    "extensions.torlauncher.start_tor": True
}

profile = FirefoxProfile(r'C:\Users\admin\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
profile.set_preference('extensions.torlauncher.start_tor', True) # bypass tor connection page
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9150)
profile.set_preference("network.proxy.socks_remote_dns", True)
profile.update_preferences()

driver = webdriver.Firefox(executable_path=geckodriverPath, 
                           firefox_binary=binary, 
                           firefox_profile=profile)



In [8]:
# Wait till your tor browser is connected to the tor network before executing this
driver.get('http://basemmnnqwxevlymli5bs36o5ynti55xojzvn246spahniugwkff2pad.onion/')


In [18]:

time.sleep(20)
    
SCROLL_PAUSE_TIME = 10 # as the Tor browser is slow, might need to increase this accordingly

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [45]:
companyNames = []
companyDescription = []
companyLocation = []
noClicks = []
dateVictimised = []
comment = []


cards = driver.find_elements_by_xpath("//div[@class='list-group-item rounded-3 py-3 bg-body-secondary text-bg-dark mb-2 position-relative']")

for card in cards:

    companyNameElement = card.find_element_by_xpath(".//a[@class='stretched-link']")
    companyNameText = companyNameElement.text
    companyNames.append(companyNameText)
    
    
    dateVictimisedElement = card.find_element_by_xpath("(.//div[@class='d-flex gap-2 small mt-1 opacity-25']/div)[2]")
    dateVictimisedText = dateVictimisedElement.text
    
    date_index = 8
    date_text = dateVictimisedText[date_index::]
    dateVictimised.append(date_text)
    
    noClicksElement = card.find_element_by_xpath("(.//div[@class='d-flex gap-2 small mt-1 opacity-25']/div)[3]")
    noClicksText = noClicksElement.text
    clicks_index = 6
    click_text = noClicksText[clicks_index::]
    noClicks.append(click_text)
    
    
    companyDescriptionElement = card.find_element_by_xpath(".//div[@class='small opacity-50']")
    companyDescriptionText = companyDescriptionElement.text
    companyDescription.append(companyDescriptionText)
    
    
    
    
    commentElement = card.find_element_by_xpath("(.//div[@class='small opacity-50'])[2]")
    commentText = commentElement.text
    
    comment_index = 30
    comment_text = commentText[comment_index::]
    new_text = comment_text.replace('\n',',')
    comment.append(new_text)
    
    
    
    

print(companyNames)
print(companyDescription)
print(companyLocation)
print(noClicks)
print(dateVictimised)
print(comment)
#driver.quit()

['KTUA Landscape Architecture and Planning', 'Comtek Advanced Structures, a Latecoere Company', 'ExdionInsurance', 'Petersen Johnson', 'Ted Pella Inc.', 'Sabian Inc', 'Chula Vista Electric (CVE)', 'C.F. Service and Supply', 'Notel', 'Muenz-Engineered Sales', 'SUD TRADING COMPANY', 'WEBBER RESTAURANT GROUP', 'Praxis Arndt und Langer', 'Springer Eubank', 'The Envelope Works Ltd', 'FabricATE Engineering', 'Araújo e Policastro Advogados', 'Ford Covesa', 'UNIVERSAL REALTY GROUP', 'Conselho Superior da Justiça do Trabalho', 'FRESH TASTE PRODUCE USA AND ASSOCIATES INC.', 'Prodegest Assessors', 'VVandA', 'Fenn Termite and Pest Control', 'Petkus Brothers', 'PRIDE GLOBAL CONSULTING SL', 'PT. Cahaya Benteng Mas', 'Shanghai FRP Research Institute Co., Ltd.', 'KLM Laboratories Pvt. Ltd', 'Varna Packaging', 'SKYROOT', 'Sydenham Laboratories', 'FA Foundry', 'INSTITUTO NACIONAL DE ELECTRIFICACION', 'HFH Capital', 'Bahamas Medical and Surgical Supplies', 'The Law Offices of Steven H. Heisler', 'Kevills

## 3) Clean data + Extract to CSV/Excel format


Requirements for data:  
- Date of victim company within 2023

In [46]:
df = pd.DataFrame({"Company": companyNames, "Company Description": companyDescription, "Date Victimised": dateVictimised, "No of views": noClicks, "Comment": comment})
df



,Company,Company Description,Date Victimised,No of views,Comment
0,KTUA Landscape Architecture and Planning,"As planners and landscape architects, KTUA is ...",17.10.2023,919,"Invoice,Receipts,Accounting documents,Personal..."
1,"Comtek Advanced Structures, a Latecoere Company","Comtek designs, manufactures and repairs compo...",17.10.2023,906,"Invoice,Receipts,Accounting documents,Personal..."
2,ExdionInsurance,Exdion is the leading Digital Insurance Platfo...,17.10.2023,2086,"Invoice,Receipts,Accounting documents,Personal..."
3,Petersen Johnson,Petersen Johnson is a leading Arizona personal...,15.10.2023,3365,"Invoice,Receipts,Accounting documents,Personal..."
4,Ted Pella Inc.,"Electron microscopy supplies and instruments, ...",10.10.2023,6548,"Invoice,Receipts,Accounting documents,Personal..."
...,...,...,...,...,...
183,Concept Fasteners,Concept Fasteners are dedicated to manufacturi...,06.06.2022,79141,S#knzaQ2bIocBsRInP60n_XA
184,MTS Office,MTS Office is the authorized Toshiba and Lexma...,15.05.2022,77688,a access!!!!!
185,ESSPEE,Market leaders in the field of resistant mater...,05.03.2022,77169,of this company and we are ready to share. wr...
186,"GIOTTO - COMÉRCIO DE VESTUÁRIO, UNIPESSOAL, LDA","GIOTTO - Comércio de Vestuário Unipessoal, Lda...",26.04.2022,78241,irectory/company-profiles.GIOTTO_-_COM%C3%89RC...


In [47]:
df.to_csv("8BASE.csv")

## 3) Scraping Glassdoor for more company information

More information we need:  
- Industry  
- Geography  
  - Country  

Additional information we can scrape (the more we have, the more insights we can infer):
- Company Website

In [ ]:
# pip install webdriver-manager
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()

In [ ]:
import requests

def location_to_country(location):
    base_url = "https://nominatim.openstreetmap.org/search"
    
    params = {
        "addressdetails": 1,
        "q": location,
        "format": "jsonv2",
        "limit": 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if data:
        country = data[0].get("address", {}).get("country", "")
        return country

    return np.nan

In [ ]:
# Using LinkedIn

companyWebsite_list = []
industry_list = []
headquarter_list = []
country_list = []

wait = WebDriverWait(driver, 5)

for company in companyNames:
    try:
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(company + ' linkedin')
        search.send_keys(Keys.RETURN)
        linkedin_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        time.sleep(2) # to limit rate of requests to linkedin
        linkedin_page.click()

        if "linkedin" in driver.current_url:

            loaded = wait.until(EC.presence_of_element_located((By.ID, "main-content")))

            # Check if the scraped company name is even in the linkedin page (sometimes the company data is not in linkedin; false positive)
            # try:
            #     linkedin_companyName_element = driver.find_element_by_xpath("//h1[@class='top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0']")
            #     linkedin_companyName = linkedin_companyName_element.text
            # except:
            #     print(company)
            #     continue

            # if not substring_included(company, linkedin_companyName):
            #     print(company)
            #     continue

            # bypass sign in modal
            try:
                sign_in_button_element = driver.find_element_by_xpath("//button[@class='sign-in-modal__outlet-btn cursor-pointer btn-md btn-primary']")
                sign_in_button_element.click()
                dismiss_button_element = driver.find_element_by_xpath("//button[@class='modal__dismiss btn-tertiary h-[40px] w-[40px] p-0 rounded-full indent-0 sign-in-modal__dismiss absolute right-0 cursor-pointer m-[20px]']")
                dismiss_button_element.click()
            except:
                pass

            CLASS = ".//dd[@class='font-sans text-md text-color-text break-words overflow-hidden']"

            try:
                website_element = driver.find_element_by_css_selector('[data-test-id="about-us__website"]')
                website_text_element = website_element.find_element_by_xpath(CLASS)
                website_text = website_text_element.text
            except Exception as e:
                website_text = np.nan
                print(e)
            companyWebsite_list.append(website_text)

            try:
                industry_element = driver.find_element_by_css_selector('[data-test-id="about-us__industry"]')
                industry_text_element = industry_element.find_element_by_xpath(CLASS)
                industry_text = industry_text_element.text
            except Exception as e:
                industry_text = np.nan
                print(e)
            industry_list.append(industry_text)
            
            try:
                headquarters_element = driver.find_element_by_css_selector('[data-test-id="about-us__headquarters"]')
                headquarters_text_element = headquarters_element.find_element_by_xpath(CLASS)
                headquarters_text = headquarters_text_element.text

                country = location_to_country(headquarters_text)
            except Exception as e:
                headquarters_text = np.nan
                country = np.nan
                print(e)
            headquarter_list.append(headquarters_text)
            country_list.append(country)

        else:
            companyWebsite_list.append(np.nan)
            industry_list.append(np.nan)
            headquarter_list.append(np.nan)
            country_list.append(np.nan)

    except Exception as e:
        companyWebsite_list.append(np.nan)
        industry_list.append(np.nan)
        headquarter_list.append(np.nan)
        country_list.append(np.nan)
        print(e)
    number += 1
    
driver.quit()

In [ ]:
df["Company Website"] = companyWebsite_list
df["Industry"] = industry_list
df["Location"] = headquarter_list
df["Country"] = country_list
df

In [ ]:
df.to_csv("Medusa_Linkedin_Data.csv")